In [1]:
sc, spark

(<pyspark.context.SparkContext at 0x104c4fc10>,
 <pyspark.sql.session.SparkSession at 0x104caaf90>)

In [2]:
import pyspark as ps
from pyspark.sql.types import *
from pyspark.ml.recommendation import *
from pyspark.ml.evaluation import *
import numpy as np
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from scipy.spatial.distance import cdist
import pandas as pd
import time
import ast

In [3]:
# sc = ps.SparkContext('local[128]')
# spark = ps.SQLContext(sc)

# from pyspark.sql import SparkSession

# spark = (SparkSession.builder
#     .master("spark://localhost:7077")
#     .config("spark.driver.cores", 127)
#     .config("spark.driver.memory", "1000g")
#     .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
#     .config("spark.driver.extraClassPath","")
#     .config("spark.driver.extraClassPath","")
#     .appName("artistrecommender")
#     .getOrCreate())

## Data frame loads

In [4]:
artist_follower_df = spark.read.csv('../data/artist_follower.csv').persist()
artist_df = spark.read.csv('../data/artist_meta.csv').persist() #manually flagged duplicates to remove after sc scrape as 1's
follower_df = spark.read.csv('../data/followers.csv').persist() 

artist_follower_df.createOrReplaceTempView('artist_follower')
artist_df.createOrReplaceTempView('artists')
follower_df.createOrReplaceTempView('followers')

## Artist & Follower column name generation

In [5]:
artist_follower_df = spark.sql("""
SELECT distinct
    /*
    _c0 as exception
    ,_c1 as artist_num
    ,_c2 as artist_follower_num
    ,*/
    _c3 as artist_alias
    ,_c4 as follower_alias
FROM artist_follower where _c2 != 'artist_follower_num'
LIMIT 10000
""").persist()

artist_follower_df.createOrReplaceTempView('artist_follower')

#0 means the artist is not a duplicate record
artist_df = spark.sql("""
SELECT distinct
    _c0 as index
    ,_c1 as id
    ,_c2 as artist_name
    ,_c3 as sc_alias
    ,_c4 as city
    ,_c5 as country
    ,_c6 as followers_count
    ,_c7 as followings_count
    ,_c8 as last_modified
    ,_c9 as playlist_count
    ,_c10 as plan
    ,_c11 as public_favorites_count
    ,_c12 as track_count
    ,_c13 as count
FROM artists where _c2 != 'artist_name'
AND _c14 = 0
""").persist()

artist_df.createOrReplaceTempView('artists')

follower_df = spark.sql("""
SELECT distinct
    --_c0 as exception
    --,_c1 as artist_num
    --,_c2 as artist_follower_num
    _c3 as artist_alias
    ,_c4 as follower_alias
    ,_c5 as comments_count
    ,_c6 as followers_count
    ,_c7 as followings_count
    --,_c8 as last_modified
    ,_c9 as likes_count
    ,_c10 as plan
    ,_c11 as playlist_count
    ,_c12 as public_favorites_count
    ,_c13 as reposts_count
    ,_c14 as track_count
    ,_c15 as uri
    ,_c16 as username
FROM followers where _c0 != 'exception'
""").persist()

follower_df.createOrReplaceTempView('followers')

## Artist & Follower id generation

In [6]:
custom_artist_id_df = spark.sql("""
select distinct
    row_number() over(order by sc_alias) as artist_id
    ,sc_alias as artist_alias
from artists
group by sc_alias
""").persist()

custom_artist_id_df.createOrReplaceTempView('custom_artist_id')

custom_follower_id_df = spark.sql("""
select distinct
    row_number() over(order by follower_alias) as follower_id
    ,follower_alias
from followers
group by follower_alias
""").persist()

custom_follower_id_df.createOrReplaceTempView('custom_follower_id')

artist_follower_ids_df = spark.sql("""
SELECT DISTINCT
    caid.artist_id
    ,cfid.follower_id
    ,af.artist_alias
    ,af.follower_alias
    ,1 as count
FROM artist_follower af
JOIN custom_artist_id caid on af.artist_alias = caid.artist_alias
JOIN custom_follower_id cfid on af.follower_alias = cfid.follower_alias
""").persist()

artist_follower_ids_df.createOrReplaceTempView('artist_follower')

## ALS model generation

In [ ]:
als_model = ALS(rank=2, implicitPrefs=True,
          userCol="follower_id", itemCol="artist_id", ratingCol="count", nonnegative=True)

In [10]:
# als_model = ALS(rank=5, maxIter=5, seed=0, regParam=1, implicitPrefs=True,
#           userCol="follower_id", itemCol="artist_id", ratingCol="count", nonnegative=True)

In [8]:
model = als_model.fit(artist_follower_ids_df)


KeyboardInterrupt


## Load model item features

In [6]:
item_features = pd.read_csv('../data/bestModelItems.csv')

In [7]:
item_features.drop('Unnamed: 0', inplace=True, axis = 1)

In [8]:
items = spark.createDataFrame(item_features)

In [9]:
items.createOrReplaceTempView('item_features')

## Combine item_features df into new artist_meta data df

In [10]:
artist_info = spark.sql("""
    SELECT distinct
        if.id
        ,if.features
        ,a.artist_name
        ,a.sc_alias
        ,a.city
        ,a.country
        ,a.followers_count
        ,a.followings_count
        ,a.last_modified
        ,a.playlist_count
        ,a.plan
        ,a.public_favorites_count
        ,a.track_count
        ,a.count
    FROM item_features if 
    JOIN custom_artist_id caid on if.id = caid.artist_id
    JOIN artists a on caid.artist_alias = a.sc_alias
""")

In [48]:
artist_meta = artist_info.toPandas()

In [49]:
artist_meta['formatted_features'] = artist_meta['features'].map(lambda x: ast.literal_eval(x))

## Convert features to matrix and calculate cosine similarity

In [13]:
items_mat = np.array(list(artist_meta['formatted_features'].values)).astype(float)

In [14]:
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine


In [15]:
cos_sim = 1-pairwise_distances(items_mat, metric="cosine")

In [18]:
artist_meta.to_excel('../data/artist_meta.xlsx')

In [55]:
artist_meta.to_json('../data/artist_meta.json')

In [44]:
artist_meta2 = pd.read_excel('../data/artist_meta.xlsx')

In [20]:
np.savetxt('../data/cos_sim_mat.txt', cos_sim)

In [21]:
mat = np.loadtxt('../data/cos_sim_mat.txt')

In [56]:
artist_meta = pd.read_json('../data/artist_meta.json')

In [57]:
items_mat = np.array(list(artist_meta['formatted_features'].values)).astype(float)

## Concatenate cos_sim matrix with meta data for easy lookup.

In [92]:
cols = list(artist_meta.columns)

In [93]:
similarity_indices = list(artist_meta['sc_alias'])

In [94]:
cols.extend(similarity_indices)

In [24]:
# 1. determine matrix index of artist. 
# 2. get the indices of the top n+1 artists - ignoring the index of the 1st b/c it will be the same artist
# 3. retrieve the aliases associated to those indices from a list
# 4. zip aliases with cosine similarities - sort zipped list by cosine similarity in descending order.
# 5. return results of aliases

# 1. Store the cos_sim matrix
# 2. Store the artist_meta table

# App:
# 1. Load the cos_sim matrix
# 2. Load the artist_meta table
# 3. Store an html string formatted with each  alias and artist name in the following format in Excel:
# sort artists by name in alphabetical order
#                <option value='choose_one'>Choose Home Team</option>
#                <option value="9">Arsenal</option>
# 3. Retrieve the user's searched alias
# 4. Retrieve aliases of the top 24 most similar artists
# 5. For each artist, populate their alias 

# Ideal App:
# 1. Allow the user to filter the recommendations by min_followers and max_followers
# 2. Allow the user to filter the artists by min_followers and max_followers

In [25]:
n = 30

In [26]:
alias = 'alephhhhh'

In [42]:
artist = artist_meta[artist_meta['sc_alias']==alias]['artist_name'].values[0]

In [43]:
artist

u'Aleph'

In [35]:
row

,id,features,artist_name,sc_alias,city,country,followers_count,followings_count,last_modified,playlist_count,plan,public_favorites_count,track_count,count,formatted_features
28,133,"[0.005527534522116184, 0.005194265395402908, 0...",Aleph,alephhhhh,Moscow,None,1907,590,2016/12/13 15:29:43 +0000,0,Free,228,14,None,"[0.00552753452212, 0.0051942653954, 0.00628775..."


In [27]:
artist_aliases = list(artist_meta['sc_alias'])

In [58]:
len(artist_aliases)

4911

In [59]:
len(set(artist_aliases))

4834

In [28]:
index = artist_aliases.index(alias)

In [29]:
arr = cos_sim[index]

In [31]:
similar_aliases = np.asarray(artist_aliases)[arr.argsort()[-(n+1):][::-1][1:]]

In [33]:
similar_aliases

array([u'becoming-real', u'nosybmols', u'headhunter', u'coolyg',
       u'callahan_uk', u'drocar', u'geiom', u'toyc', u'alismusic',
       u'babastiltz', u'samoyed', u'heterotic', u'cuppcave',
       u'sapphire-slows', u'egyptrixx', u'ivvvo', u'thefft', u'bazzerk',
       u'jay-weed', u'airbird', u'boxworkmusic', u'hyetal', u'peverelist',
       u'punchdrunkrecords', u'aisha-devi', u'mikaelseifu',
       u'benedek_music', u'damu', u'brackles', u'ghostsontape'], 
      dtype='<U27')

In [32]:
artist_meta[artist_meta['sc_alias'].isin(similar_aliases)]

,id,features,artist_name,sc_alias,city,country,followers_count,followings_count,last_modified,playlist_count,plan,public_favorites_count,track_count,count,formatted_features
110,2335,"[0.00670914351940155, 0.006264227908104658, 0....",IVVVO,ivvvo,London,None,6866,19,2016/11/27 18:04:32 +0000,0,Free,577,2,None,"[0.0067091435194, 0.0062642279081, 0.007467627..."
287,937,"[0.006108156405389309, 0.005767497234046459, 0...",Cooly G,coolyg,London,None,2983,2,2016/11/07 22:11:42 +0000,1,Free,9,7,None,"[0.00610815640539, 0.00576749723405, 0.0070297..."
288,1032,"[0.0068186623975634575, 0.006455729249864817, ...",Damu,damu,Sheffield,Britain (UK),7213,215,2016/09/21 18:47:01 +0000,4,Pro Plus,1,22,None,"[0.00681866239756, 0.00645572924986, 0.0077431..."
1305,107,"[0.005220510996878147, 0.004952860064804554, 0...",Airbird,airbird,Brooklyn,United States,1555,34,2016/10/13 14:35:01 +0000,4,Pro Plus,7,33,None,"[0.00522051099688, 0.0049528600648, 0.00605716..."
1349,5338,"[0.005483920685946941, 0.005203558132052422, 0...",Toyc,toyc,London,None,1757,122,2016/10/24 12:56:17 +0000,2,Pro,124,3,None,"[0.00548392068595, 0.00520355813205, 0.0062982..."
1678,631,"[0.006880370434373617, 0.006514685694128275, 0...",Brackles,brackles,London,None,7759,369,2016/11/23 20:41:51 +0000,2,Free,64,9,None,"[0.00688037043437, 0.00651468569413, 0.0078168..."
1985,486,"[0.0063577620312571526, 0.006131819449365139, ...",Benedek,benedek_music,"CAFE BENE , LOS ANGELES",United States,5124,416,2016/10/04 01:33:09 +0000,0,Free,459,3,None,"[0.00635776203126, 0.00613181944937, 0.0072976..."
2274,995,"[0.005722653586417437, 0.005387454759329557, 0...",Cupp Cave,cuppcave,None,None,2373,59,2014/12/15 14:08:02 +0000,0,Free,7,0,None,"[0.00572265358642, 0.00538745475933, 0.0065835..."
2303,5153,"[0.006321607157588005, 0.005954438354820013, 0...",Thefft,thefft,London,None,3558,284,2016/09/16 04:52:01 +0000,0,Pro,602,13,None,"[0.00632160715759, 0.00595443835482, 0.0070958..."
2313,469,"[0.0057228561490774155, 0.005356927402317524, ...",Becoming Real,becoming-real,norwegianway,None,2142,79,2016/09/15 14:34:31 +0000,8,Free,8,26,None,"[0.00572285614908, 0.00535692740232, 0.0065645..."


In [101]:
test = pd.concat([artist_meta, pd.DataFrame(cos_sim)], axis=1)

In [103]:
test.columns = cols

In [104]:
test

,id,features,artist_name,sc_alias,city,country,followers_count,followings_count,last_modified,playlist_count,...,pi-ge,silky1122,studio-of-scott-grooves,studio_ost,subspec-music,tonibraxton,undergroundquality,undergroundquality,viroid-life,zero7
0,220,"[0.011487905867397785, 0.004812932573258877, 0...",Amnesys,amnesys,Roma,Italy,7222,6,2016/12/05 11:20:43 +0000,3,...,0.862838,0.886397,0.880531,0.879552,0.865990,0.795705,0.882244,0.882244,0.867402,0.913396
1,745,"[0.005809524562209845, 0.005388314835727215, 0...",Casbah 73,casbah-73,Madrid,Spain,2284,315,2016/12/08 08:57:22 +0000,7,...,0.996808,0.999701,0.999738,0.999389,0.997517,0.909610,0.999701,0.999701,0.997597,0.988378
2,1177,"[0.007013583555817604, 0.0065416391007602215, ...",Delroy Edwards,delroy-edwards,LOS ANGELES,United States,16152,75,2015/07/31 22:52:08 +0000,0,...,0.995946,0.999524,0.999625,0.999863,0.997517,0.921426,0.999630,0.999630,0.998438,0.988706
3,1325,"[0.0043734596110880375, 0.004842094145715237, ...",[sic!],djavisic,Chicago,United States,905,276,2016/10/26 15:41:59 +0000,0,...,0.975442,0.987273,0.987368,0.987902,0.979087,0.900820,0.987602,0.987602,0.981531,0.996803
4,1374,"[0.003226189874112606, 0.003057810477912426, 0...",Djinji Brown,djinjibrown1,Miami,United States,573,0,2015/11/07 15:26:29 +0000,0,...,0.997771,0.998074,0.998252,0.997810,0.997535,0.904152,0.997989,0.997989,0.997495,0.981875
5,1958,"[0.005887952633202076, 0.0056527527049183846, ...",Georgia,georgia_hb,London,None,2426,0,2016/08/11 14:17:45 +0000,3,...,0.991751,0.998104,0.997937,0.998443,0.993854,0.917182,0.997960,0.997960,0.995811,0.993879
6,2097,"[0.004784272518008947, 0.0030751442536711693, ...",SCM,hardcorescm,Willenhall,Britain (UK),516,42,2016/12/23 08:37:19 +0000,9,...,0.937204,0.948330,0.946697,0.944717,0.939919,0.867419,0.948127,0.948127,0.936125,0.949732
7,2236,"[0.005387980490922928, 0.004920941311866045, 0...",Ian O'Brien,ian-obrien,新宿区 Tokyo,Japan,1709,62,2016/12/01 10:05:27 +0000,4,...,0.998186,0.999738,0.999973,0.999783,0.999038,0.918029,0.999946,0.999946,0.998947,0.984625
8,2408,"[0.007283162325620651, 0.008462832309305668, 0...",Jaques Raup̩,jaquesraupe,Bülstringen,None,14304,16,2016/10/16 16:48:47 +0000,6,...,0.980339,0.988371,0.989449,0.989078,0.983378,0.900282,0.989747,0.989747,0.983100,0.989973
9,2479,"[0.0068942042998969555, 0.006244171876460314, ...",The Persuader,jesperdahlback,Stockholm,Sweden,5601,4,2016/06/17 07:08:53 +0000,1,...,0.997627,0.999727,0.999923,0.999563,0.998533,0.916003,0.999966,0.999966,0.998139,0.985818


## Follower overlap analysis

In [11]:
artist_follower_group = artist_follower.groupby('follower_alias')

In [29]:
follower_count = artist_follower_group.count().sort_values(by='artist_alias', ascending=False)

In [30]:
follower_count = follower_count['artist_alias']

In [31]:
follower_count = pd.DataFrame(follower_count)

In [34]:
follower_count.reset_index(inplace=True)

In [37]:
follower_count.columns = ['follower_alias', 'count']

In [46]:
#6392743 unique followers
#29125208 total followers
#26640 followers that were following more than 100 of the ~5000 artists scraped
#504836 followers following more than 10 of the ~5000 artists scraped